# Import

In [1]:
import optuna
from optuna.visualization import plot_edf, plot_slice, plot_optimization_history, plot_param_importances

# DeterministicProcess params

In [ ]:
df_full = pd.concat([df_train, df_test], axis=0)
    
def objective(trial):
    
    fourier_order = trial.suggest_int("fourier_order", 0, 20)
    constant = trial.suggest_categorical("constant", [True, False])
    dp_order = trial.suggest_int("dp_order", 1, 3)
    seasonal = trial.suggest_categorical("seasonal", [True, False])
    
    #     Prepare datasets with new params
    df_full_prep = data_prep_optuna(df_full, fourier_order, constant, dp_order, seasonal)
    features = df_full_prep.columns
    df_full_prep['date'] = df_full.date
    df_full_prep['num_sold'] = df_full.num_sold.astype(np.float32)

    df_train_prep = df_full_prep[df_full_prep.date.dt.year < 2018]
    df_val_prep = df_full_prep[df_full_prep.date.dt.year == 2018]
    df_test_prep = df_full_prep[df_full_prep.date.dt.year == 2019]
    
    #     Eval model
    preproc, model, y_va_pred = fit_simple_model(df_train_prep, X_va=df_val_prep, outliers=False, features=features)
    y_va_true = df_val_prep.num_sold.values.reshape(-1, 1)
    score = f_smape(y_va_true, y_va_pred)

    return score

optuna.logging.set_verbosity(optuna.logging.INFO)
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)

df_study = study.trials_dataframe()
df_study.to_excel('study_DeterministicProcess_params.xlsx')

best_params = study.best_params
best_params

# Models optimization

## ExtraTreesRegressor

In [ ]:
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int("n_estimators", 50,200),
#         'criterion': trial.suggest_categorical("criterion", ['squared_error', 'absolute_error']),
#         'max_depth': trial.suggest_int("max_depth", 3,15),
#         'min_samples_split': trial.suggest_int("min_samples_split", 2, 5),
#         'min_samples_leaf': trial.suggest_int("min_samples_leaf", 1, 5),
        'max_features': trial.suggest_categorical("max_features", ['auto', 'sqrt', 'log2']),
#         'bootstrap': trial.suggest_categorical("bootstrap", [True, False]),
        'n_jobs': -1,
        'random_state': 0,
        }
    
    model = ExtraTreesRegressor(**params)
    score = modelisation_TT_scoring(model, preprocessor, X_train, y_train)
    return score

optuna.logging.set_verbosity(optuna.logging.INFO)
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

df_study = study.trials_dataframe()
df_study.to_excel('study_ET.xlsx')

ET_params = study.best_params
ET_params

## NuSVR

In [ ]:
def objective(trial):
    # default params : nu=0.5, C=1.0, kernel='rbf', degree=3, gamma='scale', coef0=0.0, shrinking=True, tol=0.001, cache_size=200, verbose=False, max_iter=- 1)
    params = {
        'nu' : trial.suggest_float("nu", 0.2, 0.5), 
        'C' : trial.suggest_float("C", 0.1, 1),
        'kernel' : trial.suggest_categorical("kernel", ['linear', 'poly', 'rbf']),
#         'kernel':'linear',
#         'degree' : trial.suggest_int("degree", 2, 4),
#         'tol' : trial.suggest_float("tol", 5e-4, 1e-2, log=True),
        'tol':5e-4,
#         'max_iter': 30000,
    }
    
    model = NuSVR(**params)
    
    score = modelisation_TT_scoring_NuSVR (model, preprocessor, X_train, y_train)
    return score

optuna.logging.set_verbosity(optuna.logging.INFO)
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

df_study = study.trials_dataframe()
df_study.to_excel('study_NuSVR.xlsx')

NuSVR_params = study.best_params
NuSVR_params

## XGBRegressor

In [36]:
def objective(trial):
    
    params = {
        'learning_rate' : trial.suggest_float("learning_rate", 0.05, 1),
        'max_depth' : trial.suggest_int("max_depth", 3, 15),
        'min_child_weight' : trial.suggest_int("min_child_weight", 1, 7),
        'gamma' : trial.suggest_float("gamma", 0, 1),
        'colsample_bytree' : trial.suggest_float("colsample_bytree", 0.3, 1),
        'colsample_bylevel' : trial.suggest_float("colsample_bylevel", 0.3, 1),
        'colsample_bynode' : trial.suggest_float("colsample_bynode", 0.3, 1),
        'n_estimators' : trial.suggest_int("n_estimators", 50, 250),
        'n_jobs' : -1,
             }
    model = XGBRegressor(**params)
    score = modelisation_TT_scoring (model, preprocessor, X_train, y_train)
    return score

optuna.logging.set_verbosity(optuna.logging.INFO)
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

df_study = study.trials_dataframe()
df_study.to_excel('study_XGB.xlsx')

XGB_params = study.best_params
XGB_params

[I 2021-12-09 18:50:58,001] A new study created in memory with name: no-name-b6c34993-ecb3-47ff-a672-64a157847ace
[I 2021-12-09 18:50:59,179] Trial 0 finished with value: -0.13202730167903123 and parameters: {'learning_rate': 0.21570924625143573, 'max_depth': 4, 'min_child_weight': 4, 'gamma': 0.25356766601922687, 'colsample_bytree': 0.5498654941263128, 'colsample_bylevel': 0.5409373765169265, 'colsample_bynode': 0.6527176738216981, 'n_estimators': 98}. Best is trial 0 with value: -0.13202730167903123.
[I 2021-12-09 18:51:02,255] Trial 1 finished with value: -0.16917173013563974 and parameters: {'learning_rate': 0.8809840370584787, 'max_depth': 8, 'min_child_weight': 7, 'gamma': 0.6040757765834368, 'colsample_bytree': 0.6905884008914027, 'colsample_bylevel': 0.3685284861816279, 'colsample_bynode': 0.5807895021503102, 'n_estimators': 197}. Best is trial 0 with value: -0.13202730167903123.
[I 2021-12-09 18:51:07,120] Trial 2 finished with value: -0.1369301119852827 and parameters: {'lear

{'learning_rate': 0.05088631222780335,
 'max_depth': 5,
 'min_child_weight': 4,
 'gamma': 0.0005094718577197503,
 'colsample_bytree': 0.4001427669533314,
 'colsample_bylevel': 0.641982498677674,
 'colsample_bynode': 0.40782719477700585,
 'n_estimators': 185}

## RandomForestRegressor

In [44]:
def objective(trial):
#     n_estimators=100, *, criterion='squared_error', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
#     max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, 
#     verbose=0, warm_start=False, ccp_alpha=0.0, max_samples=None    
    params = {
        'n_estimators' : trial.suggest_int("n_estimators", 50, 200),
    #     'criterion' : trial.suggest_categorical("criterion", ["squared_error", "absolute_error", "poisson"]),
    #     'max_depth' : trial.suggest_int("max_depth", 3, 15),
    #     'min_samples_split' : trial.suggest_int("min_samples_split", 2, 5),
    #     'min_samples_leaf' : trial.suggest_int("min_samples_leaf", 1, 5),
        'n_jobs':-1,
             }
    
    model = RandomForestRegressor(**params)
    
    score = modelisation_TT_scoring (model, preprocessor, X_train, y_train)
    return score

optuna.logging.set_verbosity(optuna.logging.INFO)
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

df_study = study.trials_dataframe()
df_study.to_excel('study_RF.xlsx')

RF_params = study.best_params
RF_params

[I 2021-12-09 20:37:34,623] A new study created in memory with name: no-name-11090871-8f56-4b94-8465-274fa1e020e6
[I 2021-12-09 20:37:42,717] Trial 0 finished with value: -0.13410019810273896 and parameters: {'n_estimators': 167}. Best is trial 0 with value: -0.13410019810273896.
[I 2021-12-09 20:37:47,558] Trial 1 finished with value: -0.13407488114678548 and parameters: {'n_estimators': 148}. Best is trial 1 with value: -0.13407488114678548.
[I 2021-12-09 20:37:50,900] Trial 2 finished with value: -0.13577137954237414 and parameters: {'n_estimators': 94}. Best is trial 1 with value: -0.13407488114678548.
[I 2021-12-09 20:37:54,805] Trial 3 finished with value: -0.13480892785793314 and parameters: {'n_estimators': 113}. Best is trial 1 with value: -0.13407488114678548.
[I 2021-12-09 20:37:58,267] Trial 4 finished with value: -0.1353063926038285 and parameters: {'n_estimators': 95}. Best is trial 1 with value: -0.13407488114678548.
[I 2021-12-09 20:38:02,923] Trial 5 finished with valu

{'n_estimators': 136}

# MLPClassifier

In [ ]:
warnings.simplefilter(action='ignore', category=UserWarning)

N_FOLDS = 3
subsample_pr = 0.8

df_mlp = pd.DataFrame(Normalizer().fit_transform(df), columns=cols)
X = df_mlp.loc[index_trusted]
y = pd.Series(pred_method_2[index_trusted])

def objective(trial):
    # hidden_layer_1 = int(trial.suggest_categorical("hidden_layer_1", [1, 2, 4, 8, 16, 32, 64, 128]))
    # hidden_layer_2 = int(trial.suggest_categorical("hidden_layer_2", [0, 1, 2, 4, 8, 16, 32, 64, 128]))
    # hidden_layer_3 = int(trial.suggest_categorical("hidden_layer_3", [0, 1, 2, 4, 8, 16, 32, 64, 128]))
    hidden_layer_1 = 2 ** trial.suggest_int("hidden_layer_1", 0, 7)
    hidden_layer_2 = 2 ** trial.suggest_int("hidden_layer_2", 0, 7)
    hidden_layer_3 = 2 ** trial.suggest_int("hidden_layer_3", 0, 7)
    if hidden_layer_2 == 1:
        hidden_layer_sizes = (hidden_layer_1)
    elif hidden_layer_3 == 1:
        hidden_layer_sizes = (hidden_layer_1, hidden_layer_2)
    else:
        hidden_layer_sizes = (hidden_layer_1, hidden_layer_2, hidden_layer_3)
        
    print(hidden_layer_sizes)
    
    params_MLP = {    
        'hidden_layer_sizes' : hidden_layer_sizes,
        # 'activation' : trial.suggest_categorical("activation", ['identity', 'logistic', 'tanh', 'relu']),
        # 'activation' : trial.suggest_categorical("activation", ['logistic', 'tanh']),
#         'activation': 'logistic',
        # 'solver' : trial.suggest_categorical("solver", ['lbfgs', 'sgd', 'adam']), #lbfgs is the best choice for small dataset
        # 'solver': 'adam',
        # 'tol' : trial.suggest_float("tol", 5e-5, 1e-3, log=True),
        # 'alpha' : trial.suggest_float("alpha", 5e-4, 1e-2, log=True),
        # 'learning_rate' : trial.suggest_categorical("learning_rate", ['constant', 'invscaling', 'adaptive']),
#         'learning_rate': 'adaptive',
        # 'learning_rate_init': trial.suggest_float("learning_rate_init", 1e-5, 1e-2, log=True),
#         'max_iter': trial.suggest_int("max_iter", 50, 500),
        'max_iter': 1000,
#         'beta_1': trial.suggest_float("beta_1", 0.1, 1),
#         'beta_2': trial.suggest_float("beta_2", 0.1, 1),
        'early_stopping': True,
        # 'random_state': 0,
        'warm_start': True,
             }

    score = 0
    for seed in range(N_FOLDS):
        subsample_X, _, subsample_y, _ = train_test_split(X, y, train_size=subsample_pr, random_state=seed)
        model = MLPClassifier(**params_MLP)
        model.fit(subsample_X, subsample_y)
        # print(model.score(subsample_X, subsample_y), model.score(X, y), model.score(df_mlp, pred_method_2))
        score += model.score(df_mlp, pd.Series(pred_method_2))
    return score / N_FOLDS

optuna.logging.set_verbosity(optuna.logging.INFO)
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

df_study = study.trials_dataframe()
df_study.to_excel('study_MLP.xlsx')

MLP_params = study.best_params
MLP_params

[I 2022-07-20 14:54:40,369] A new study created in memory with name: no-name-ae39371e-0383-4b83-a9c6-32261c247d5c
[I 2022-07-20 14:55:09,232] Trial 0 finished with value: 0.5735884353741496 and parameters: {'hidden_layer_1': 2}. Best is trial 0 with value: 0.5735884353741496.


## MLPRegressor

In [49]:
def objective(trial):
    params = {    
        # 'hidden_layer_sizes' : trial.suggest_int("hidden_layer_sizes", 50, 300),
        'hidden_layer_sizes': 250,
        # 'activation' : trial.suggest_categorical("activation", ['identity', 'logistic', 'tanh', 'relu']),
        'activation' : trial.suggest_categorical("activation", ['logistic', 'tanh']),
#         'activation': 'logistic',
        # 'solver' : trial.suggest_categorical("solver", ['lbfgs', 'sgd', 'adam']), #lbfgs is the best choice for small dataset
        'solver': 'adam',
        'tol' : trial.suggest_float("tol", 5e-5, 1e-3, log=True),
        'alpha' : trial.suggest_float("alpha", 5e-4, 1e-2, log=True),
        'learning_rate' : trial.suggest_categorical("learning_rate", ['constant', 'invscaling', 'adaptive']),
#         'learning_rate': 'adaptive',
        'learning_rate_init': trial.suggest_float("learning_rate_init", 1e-5, 1e-2, log=True),
#         'max_iter': trial.suggest_int("max_iter", 50, 500),
        'max_iter': 1000,
#         'beta_1': trial.suggest_float("beta_1", 0.1, 1),
#         'beta_2': trial.suggest_float("beta_2", 0.1, 1),
        'early_stopping': True,
        'random_state': 0,
        'warm_start': True,
             }
    
    model = MLPRegressor(**params)
    score = modelisation_TT_scoring (model, preprocessor, X_train, y_train)
    return score

optuna.logging.set_verbosity(optuna.logging.INFO)
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

df_study = study.trials_dataframe()
df_study.to_excel('study_MLP.xlsx')

MLP_params = study.best_params
MLP_params

[I 2021-12-09 20:38:20,444] A new study created in memory with name: no-name-dc109295-3ab5-4cad-83b5-d66729600ac7
[I 2021-12-09 20:38:40,879] Trial 0 finished with value: -0.138253920040585 and parameters: {'tol': 0.00013934753187790918, 'learning_rate': 'invscaling', 'activation': 'logistic'}. Best is trial 0 with value: -0.138253920040585.
[I 2021-12-09 20:38:52,627] Trial 1 finished with value: -0.12615039510472453 and parameters: {'tol': 0.00019903196784587794, 'learning_rate': 'adaptive', 'activation': 'logistic'}. Best is trial 1 with value: -0.12615039510472453.
[I 2021-12-09 20:39:16,800] Trial 2 finished with value: -0.20110199765933773 and parameters: {'tol': 7.709685022809601e-05, 'learning_rate': 'constant', 'activation': 'relu'}. Best is trial 1 with value: -0.12615039510472453.
[I 2021-12-09 20:39:45,915] Trial 3 finished with value: -0.15115473732533063 and parameters: {'tol': 8.667589815857424e-05, 'learning_rate': 'invscaling', 'activation': 'tanh'}. Best is trial 1 wi

{'tol': 0.00030103868887435085,
 'learning_rate': 'invscaling',
 'activation': 'logistic'}

## LGBMRegressor

In [54]:
def objective(trial):
    
    params = {
        'boosting_type': trial.suggest_categorical("boosting_type", ['gbdt', 'dart', 'goss', 'rf']), # default=gbdt
        'num_leaves': trial.suggest_int("num_leaves", 7, 50), # default=31
#         'max_depth' : trial.suggest_int("max_depth", 3, 15), # default=-1
        'learning_rate' : trial.suggest_float("learning_rate", 0.01, 0.5), # default=0.1
#         'n_estimators' : trial.suggest_int("n_estimators", 50, 250), # default=100
#         'min_child_weight' : trial.suggest_float("min_child_weight", 1e-4, 1e-2, log=True), # default=1e-3
        'colsample_bytree' : trial.suggest_float("colsample_bytree", 0.3, 1), # default=1.
        'random_state': 0,
        'n_jobs' : -1,
             }
    model = LGBMRegressor(**params)
    score = modelisation_TT_scoring (model, preprocessor, X_train, y_train)
    return score

optuna.logging.set_verbosity(optuna.logging.INFO)
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

df_study = study.trials_dataframe()
df_study.to_excel('study_LGBM.xlsx')

LGBM_params = study.best_params
LGBM_params

[I 2021-12-09 20:50:21,497] A new study created in memory with name: no-name-b5948488-9bee-4b44-be76-0981b684cabb
[I 2021-12-09 20:50:21,861] Trial 0 finished with value: -1.0 and parameters: {'boosting_type': 'rf', 'num_leaves': 24, 'learning_rate': 0.3908496277582922, 'colsample_bytree': 0.9056274956289145}. Best is trial 0 with value: -1.0.
[I 2021-12-09 20:50:23,226] Trial 1 finished with value: -3.1697189239119323 and parameters: {'boosting_type': 'dart', 'num_leaves': 42, 'learning_rate': 0.043466084283251265, 'colsample_bytree': 0.4924781340339136}. Best is trial 0 with value: -1.0.
[I 2021-12-09 20:50:24,186] Trial 2 finished with value: -0.12628675270016154 and parameters: {'boosting_type': 'goss', 'num_leaves': 42, 'learning_rate': 0.07021228198531274, 'colsample_bytree': 0.709040243413522}. Best is trial 2 with value: -0.12628675270016154.
[I 2021-12-09 20:50:25,490] Trial 3 finished with value: -0.8637301184917918 and parameters: {'boosting_type': 'dart', 'num_leaves': 28, 

{'boosting_type': 'gbdt',
 'num_leaves': 16,
 'learning_rate': 0.09110805968064632,
 'colsample_bytree': 0.47467229178342357}

## MLExtend stacking regressor

In [ ]:
def evaluate_model(model, X_tr, X_va):
    
    # Preprocess the data
    X_tr_f = X_tr[features]
    # preproc = MinMaxScaler()
    preproc = StandardScaler()
    X_tr_f = preproc.fit_transform(X_tr_f)
    y_tr = X_tr.num_sold.values.reshape(-1, 1)
    
    # Train the model
    model.fit(X_tr_f, np.log(y_tr).ravel())
    y_tr_pred = np.ceil(np.exp(model.predict(X_tr_f)).reshape(-1, 1))
    score_tr = f_smape(y_tr, y_tr_pred)  
    
    y_va_pred = None

    if X_va is not None:
        # Preprocess the validation data
        X_va_f = X_va[features]
        X_va_f = preproc.transform(X_va_f)
        y_va = X_va.num_sold.values.reshape(-1, 1)

        # Inference for validation
        y_va_pred = np.ceil(np.exp(model.predict(X_va_f)).reshape(-1, 1))
        
        # Print SMAPE and plot graph
        # eval_plot (y_va, y_va_pred)
    
    #     Eval model
    y_va_true = X_va.num_sold.values.reshape(-1, 1)
    score_val = f_smape(y_va_true, y_va_pred)  
    
    return score_tr, score_val, y_tr_pred, y_va_pred

    
def objective(trial):
    models = {
    'lr' : LinearRegression(),
    'ridge' : Ridge(),
    'huber' : HuberRegressor(),
    'sgdr' : SGDRegressor(),
    'mlp' : MLPRegressor(solver = 'lbfgs', activation = 'identity'),
    'br' : BayesianRidge(),
    'lgb' : LGBMRegressor(),
    }
    
    models_selection = {    
        'lr' : trial.suggest_categorical("lr", [False, True]),
        'ridge' : trial.suggest_categorical("ridge", [False, True]),
        'huber' : trial.suggest_categorical("huber", [False, True]),
        'sgdr' : trial.suggest_categorical("sgdr", [False, True]),
        'mlp' : trial.suggest_categorical("mlp", [False, True]),
        'br' : trial.suggest_categorical("br", [False, True]),
    }
    
    meta_models = {
    'lr' : LinearRegression(),
    'ridge' : Ridge(),
    'huber' : HuberRegressor(),
    'sgdr' : SGDRegressor(),
    'mlp' : MLPRegressor(solver = 'lbfgs', activation = 'identity'),
    'br' : BayesianRidge(),
    'lgb' : LGBMRegressor(),
    }
    
    regressors = []
    for key, value in models_selection.items():
        if value:
            regressors.append(models[key])
    
    if len(regressors) == 0:
        regressors = [LinearRegression()]
            
    use_features_in_secondary = trial.suggest_categorical("use_features_in_secondary", [False, True])
    meta_regressor_selection = trial.suggest_categorical(
        "meta_regressor_selection",
        ['lr', 'ridge', 'huber', 'sgdr', 'mlp', 'br', 'lgb']
    )
                                    
    model = mlxtend.regressor.StackingRegressor(
        regressors=regressors, 
        meta_regressor=meta_models[meta_regressor_selection], 
        use_features_in_secondary=use_features_in_secondary,
    )
    score_tr, score_eval, pred_tr, y_va_pred = evaluate_model(model, df_train_prep, df_val_prep)

    score = score_tr + score_eval
    return score

optuna.logging.set_verbosity(optuna.logging.INFO)
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

df_study = study.trials_dataframe()
df_study.to_excel('study_MLP.xlsx')

MLP_params = study.best_params
MLP_params

[I 2022-01-12 10:51:55,768] A new study created in memory with name: no-name-3f5b95f7-a174-4524-b556-c7423d73159a
[I 2022-01-12 10:51:58,533] Trial 0 finished with value: 9.070035636956018 and parameters: {'lr': True, 'ridge': True, 'huber': True, 'sgdr': True, 'mlp': True, 'br': False, 'use_features_in_secondary': True, 'meta_regressor_selection': 'lgb'}. Best is trial 0 with value: 9.070035636956018.
[I 2022-01-12 10:52:00,170] Trial 1 finished with value: 8.55276807820502 and parameters: {'lr': False, 'ridge': False, 'huber': True, 'sgdr': False, 'mlp': False, 'br': False, 'use_features_in_secondary': False, 'meta_regressor_selection': 'mlp'}. Best is trial 1 with value: 8.55276807820502.
[I 2022-01-12 10:52:02,795] Trial 2 finished with value: 8.569118440679402 and parameters: {'lr': False, 'ridge': False, 'huber': False, 'sgdr': True, 'mlp': True, 'br': False, 'use_features_in_secondary': True, 'meta_regressor_selection': 'huber'}. Best is trial 1 with value: 8.55276807820502.
[I 

## Feature selection

In [ ]:
bad_features = ['row_id','time',TARGET, 'test']
POSSIBLE_FEATURES = [feat for feat in df.columns if feat not in bad_features]

def objective(trial):
    ###################################
    # Generate our trial model.
    ###################################
    FEATURES = []
    for feat in POSSIBLE_FEATURES:
        select_feat = trial.suggest_categorical(feat, [True, False])
        if select_feat:
            FEATURES.append(feat)
    model = LGBMRegressor()
    only_test = trial.suggest_categorical('only_test_day',[True,False])
    if only_test:
        msk = train.time.dt.weekday.isin([0])
    else:
        msk = [True for i in range(train.shape[0])]
    model.fit(train.loc[msk, FEATURES], train.loc[msk, TARGET])
    
    
    #Val Score
    val_preds = model.predict(val[FEATURES])
    score = np.mean(np.abs(val[TARGET].values - val_preds))    
    return score

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100)

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

# Plot

In [ ]:
plot_optimization_history(study)

In [ ]:
plot_param_importances(study)

In [ ]:
optuna.visualization.plot_param_importances(
    study, target=lambda t: t.duration.total_seconds(), target_name="duration"
)

In [ ]:
plot_slice(study)